In [42]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt

FinancialLabel = "Financial/Job Problem"
Legallabel = "Legal Problem"
SchoolLabel = "School or Academic Related Problem"
HealthLabel = "Lack of Access to Health/Mental Health Care"
FinancialPredicted = "Financial - Predicted"
LegalPredicted = "Legal - Predicted"
SchoolPredicted = "School - Predicted"
HealthPredicted = "Health - Predicted"

In [43]:

#scores = pd.read_csv('ben_dataset_all_class_scores_embeddings_using_first_384_words.csv')
#bendata = pd.read_csv('/Users/minijain/Downloads/SocWebLab/Structural_Violence/Final Ben dataset labelled.csv')
bendata = pd.read_csv('/Users/minijain/Downloads/SocWebLab/Structural_Violence/finalbendatasetwithpreds.csv')
nelagt = pd.read_csv('Transfer/labels_NELAGT.csv')
countydata = pd.read_csv('Transfer/County_data_NELALocal.csv')
publishersdata = pd.read_csv('publishers_regrouped.csv')
publishersdata['All Publishers'] = publishersdata['All Publishers'].str.strip()
publishersdata['Regional Publishers'] = publishersdata['Regional Publishers'].str.strip()
publishersdata['National Publishers'] = publishersdata['National Publishers'].str.strip()
publishersdata['Right Leaning Publishers'] = publishersdata['Right Leaning Publishers'].str.strip()
publishersdata['Left Leaning Publishers'] = publishersdata['Left Leaning Publishers'].str.strip()
publishersdata['Fringe/Questionable Publishers'] = publishersdata['Fringe/Questionable Publishers'].str.strip()

In [44]:
result = bendata
print(result.shape)

(22021, 29)


In [45]:
result.loc[result['Publisher'] == "chicagosuntimes", 'Publisher'] = "chicagosun-times"
result.loc[result['Publisher'] == "wakingtimes", 'Publisher'] = "thewakingtimes"
requiredpublishers = result['Publisher'].unique()
allregionalpublishers = countydata['source']
allnationalpublishers = nelagt['source']

common_elements = set(allnationalpublishers).intersection(allregionalpublishers)
all_elements = set(allnationalpublishers).union(allregionalpublishers)

for i in requiredpublishers:
    if i not in all_elements:
        print(i)

hitandrun
pamelagellerreport
skynewsus
bonginoreport
freedom-bunker
csmonitor


In [46]:
print(result.shape)
print("Financial: ", result[result[FinancialPredicted] > 0].shape[0])
print("Legal: ", result[result[LegalPredicted] > 0].shape[0])
print("School: ", result[result[SchoolPredicted] > 0].shape[0])
print("Health: ", result[result[HealthPredicted] > 0].shape[0])
print("Not any category: ",result[result['None - Predicted'] > 0].shape[0])
print("Multi label: ",result[(result[HealthPredicted] + result[FinancialPredicted] + result[LegalPredicted] + result[SchoolPredicted] + result['None - Predicted'] > 0)].shape[0])
print("Not Suicide: ", result[result['Reasoning'] == 'NotSuicide'].shape[0])
print("Token size exceeded: ", result[result['Reasoning'] == 'TooManyTokens'].shape[0])

(22021, 29)
Financial:  810
Legal:  4579
School:  834
Health:  3756
Not any category:  4373
Multi label:  13629
Not Suicide:  8033
Token size exceeded:  359


In [39]:
print("Not any category: ",result[result['None - Predicted'] > 0].shape[0])

Not any category:  4373


In [16]:
result = result[(result['Reasoning'] != "TooManyTokens") & (result['Reasoning'] != "NotSuicide")]
result = result.drop([FinancialLabel, SchoolLabel, Legallabel, HealthLabel, 'Unnamed: 0.3', 'Unnamed: 0.2', 'Unnamed: 0.1', 'Unnamed: 0', 'Comments', 'None', 'Error'], axis = 1)
print(result.shape)

(13629, 18)


In [8]:
result.to_csv('final working dataset with only suicide articles.csv')

### BREAK

In [47]:
nationalsources = nelagt[nelagt['country'] == 'USA']['source'].unique()
regionalsources = countydata['source'].unique()
morenational = publishersdata['National Publishers'].dropna()
moreregional = publishersdata['Regional Publishers'].dropna()
nationalsources = set(nationalsources).union(set(morenational))
regionalsources = set(regionalsources).union(set(moreregional))
print(len(nationalsources))
print(len(regionalsources))

common_elements = [element for element in nationalsources if element in regionalsources]
print(common_elements)

213
315
['dailyherald', 'chicagosun-times']


In [48]:
def national_func(row):
    val = row['Publisher']
    #print(val)
    if row['Publisher'] in nationalsources:
        return 1
    else:
        return 0
def regional_func(row):
    val = row['Publisher']
    #print(val)
    if row['Publisher'] in regionalsources:
        return 1
    else:
        return 0

# Apply the function to create the new column
result['National'] = result.apply(national_func, axis=1)
result['Regional'] = result.apply(regional_func, axis=1)

In [49]:
print("National articles: ", len(result[result['National'] == 1]))
print("Regional articles: ", len(result[result['Regional'] == 1]))
print("None articles: ", len(result[(result['National'] == 0) & (result['Regional'] == 0)]))

National articles:  14498
Regional articles:  7066
None articles:  865


In [12]:
regionaldf = result[result['Regional'] == 1]
nationaldf = result[result['National'] == 1]

In [13]:
regionaldf.to_csv('regional.csv')
nationaldf.to_csv('national.csv')

In [14]:
LabelArray = [FinancialLabel, Legallabel, SchoolLabel, HealthLabel]

In [15]:
print("National: ", nationaldf.shape[0])
print("Financial: ", nationaldf[nationaldf[FinancialPredicted] > 0].shape[0])
print("Legal: ", nationaldf[nationaldf[LegalPredicted] > 0].shape[0])
print("School: ", nationaldf[nationaldf[SchoolPredicted] > 0].shape[0])
print("Health: ", nationaldf[nationaldf[HealthPredicted] > 0].shape[0])
print("Not any category: ",nationaldf[nationaldf['None - Predicted'] > 0].shape[0])

National:  8891
Financial:  520
Legal:  3089
School:  527
Health:  2266
Not any category:  2904


In [16]:
print("National: ", nationaldf.shape[0])
print("Financial %: ", 100 * nationaldf[nationaldf[FinancialPredicted] > 0].shape[0]/nationaldf.shape[0])
print("Legal %: ", 100 * nationaldf[nationaldf[LegalPredicted] > 0].shape[0]/nationaldf.shape[0])
print("School %: ", 100 * nationaldf[nationaldf[SchoolPredicted] > 0].shape[0]/nationaldf.shape[0])
print("Health %: ", 100 * nationaldf[nationaldf[HealthPredicted] > 0].shape[0]/nationaldf.shape[0])
print("Not any category %: ", 100 * nationaldf[nationaldf['None - Predicted'] > 0].shape[0]/nationaldf.shape[0])

National:  8891
Financial %:  5.848610954898212
Legal %:  34.742998537847264
School %:  5.927342256214149
Health %:  25.4864469688449
Not any category %:  32.66224271735463


In [17]:
print("Regional: ", regionaldf.shape[0])
print("Financial: ", regionaldf[regionaldf[FinancialPredicted] > 0].shape[0])
print("Legal: ", regionaldf[regionaldf[LegalPredicted] > 0].shape[0])
print("School: ", regionaldf[regionaldf[SchoolPredicted] > 0].shape[0])
print("Health: ", regionaldf[regionaldf[HealthPredicted] > 0].shape[0])
print("Not any category: ",regionaldf[regionaldf['None - Predicted'] > 0].shape[0])

Regional:  4426
Financial:  274
Legal:  1317
School:  283
Health:  1453
Not any category:  1385


In [18]:
print("Regional: ", regionaldf.shape[0])
print("Financial %: ", 100 * regionaldf[regionaldf[FinancialPredicted] > 0].shape[0]/regionaldf.shape[0])
print("Legal %: ", 100 * regionaldf[regionaldf[LegalPredicted] > 0].shape[0]/regionaldf.shape[0])
print("School %: ", 100 * regionaldf[regionaldf[SchoolPredicted] > 0].shape[0]/regionaldf.shape[0])
print("Health %: ", 100 * regionaldf[regionaldf[HealthPredicted] > 0].shape[0]/regionaldf.shape[0])
print("Not any category %: ", 100 * regionaldf[regionaldf['None - Predicted'] > 0].shape[0]/regionaldf.shape[0])

Regional:  4426
Financial %:  6.1906913691821055
Legal %:  29.75598734749209
School %:  6.394035246272029
Health %:  32.82873926796204
Not any category %:  31.29236330772707


In [50]:
leftsources = nelagt[(nelagt['country'] == 'USA') &  ((nelagt['bias'] == 'left-center') | (nelagt['bias'] == 'left') | (nelagt['bias'] == 'left-bias') )]['source'].unique()
rightsources = nelagt[(nelagt['country'] == 'USA') &  ((nelagt['bias'] == 'right-center') | (nelagt['bias'] == 'right') | (nelagt['bias'] == 'right-bias') )]['source'].unique()
leftsources = set(leftsources).union(set(publishersdata['Left Leaning Publishers'].dropna()))
rightsources = set(rightsources).union(set(publishersdata['Right Leaning Publishers'].dropna()))
print(len(leftsources))
print(len(rightsources))
def left_func(row):
    val = row['Publisher']
    #print(val)
    if row['Publisher'] in leftsources:
        return 1
    else:
        return 0
def right_func(row):
    val = row['Publisher']
    #print(val)
    if row['Publisher'] in rightsources:
        return 1
    else:
        return 0

# Apply the function to create the new column
result['Left'] = result.apply(left_func, axis=1)
result['Right'] = result.apply(right_func, axis=1)

58
39


In [51]:
print("Left articles: ", len(result[result['Left'] == 1]))
print("Right articles: ", len(result[result['Right'] == 1]))
#print("None articles: ", len(result[(result['National'] == 0) & (result['Regional'] == 0)]))

Left articles:  9058
Right articles:  4184


In [20]:
policticalleaningadded = pd.merge(nationaldf, nelagt, left_on='Publisher', right_on='source', how='inner')
leftbias = policticalleaningadded[(policticalleaningadded['bias'] == 'left') | (policticalleaningadded['bias'] == 'left-center')]
rightbias = policticalleaningadded[(policticalleaningadded['bias'] == 'right') | (policticalleaningadded['bias'] == 'right-center')]
nobias = policticalleaningadded[policticalleaningadded['bias'] == 'center']

In [21]:
leftbias.to_csv('leftbias.csv')
rightbias.to_csv('rightbias.csv')

In [22]:
print(len(result[result['Left'] == 1]) == len(leftbias))
print(len(result[result['Right'] == 1]) == len(rightbias))

True
True


In [23]:
result.to_csv('final working dataset with only suicide articles.csv')

In [159]:
# leftbias = leftbias[leftbias['Financial - Predicted'] + leftbias['Legal - Predicted'] + leftbias['Health - Predicted'] + leftbias['School - Predicted'] > 0 ]
# rightbias = rightbias[rightbias['Financial - Predicted'] + rightbias['Legal - Predicted'] + rightbias['Health - Predicted'] + rightbias['School - Predicted'] > 0 ]
# nationaldf = nationaldf[nationaldf['Financial - Predicted'] + nationaldf['Legal - Predicted'] + nationaldf['Health - Predicted'] + nationaldf['School - Predicted'] > 0 ]
# regionaldf = regionaldf[regionaldf['Financial - Predicted'] + regionaldf['Legal - Predicted'] + regionaldf['Health - Predicted'] + regionaldf['School - Predicted'] > 0 ]

In [160]:


publishersleft = leftbias['Publisher'].unique()
publishersright = rightbias['Publisher'].unique()

publishersregional = regionaldf['Publisher'].unique()
publishersnational = nationaldf['Publisher'].unique()
combined = list(set(publishersleft).union(set(publishersright), set(publishersnational), set(publishersregional)))
df = pd.DataFrame()
# df['National Publishers'] = publishersnational
# df['Regional Publishers'] = publishersregional
# df['Left Leaning Publishers'] = publishersleft
# df['Right Leaning Publishers'] = publishersright

df['All Publishers used in analyses'] = list(set(publishersleft).union(set(publishersright), set(publishersnational), set(publishersregional)))

In [161]:
list(set(combined))

['dailyworld',
 'peninsuladailynews',
 'greenvilleadvocate',
 'campbellreporter',
 'centralkitsapreporter',
 'keizertimes',
 'theseattletimes',
 'thestatesboroherald',
 'valleymorningstar',
 'palmerreport',
 'mercerislandreporter',
 'talkingpointsmemo',
 'hotair',
 'therobesonian',
 'bossierpresstribune',
 'thenatchezdemocrat',
 'slate',
 'mountainxpress',
 'freebeacon',
 'rightwingwatch',
 'thetennesseestar',
 'theheraldnet',
 'longislandbusinessnews',
 'fayettevillefreeweekly',
 'newswars',
 'oann',
 'theconservativetreehouse',
 'thesampsonindependent',
 'sequimgazette',
 'southernstandard',
 'murrysvillestar',
 'moab’stimesindependent',
 'recordherald',
 'juneauempire',
 'thejamestownsun',
 'theblaze',
 'lagrangedailynews',
 'infowars',
 'thebitterrootstar',
 'rawstory',
 'thecoastalcourier',
 'theatlantic',
 'themonitor',
 'thecolumbian',
 'startribune',
 'newsleader',
 'duluthnewstribune',
 'grandforksherald',
 'peninsulaclarion',
 'vashonmauryislandbeachcomber',
 'southwhidbeyrec

In [162]:
policticalleaningadded[policticalleaningadded['bias'] != None]

,Unnamed: 0.7,Unnamed: 0.6,Unnamed: 0.5,Unnamed: 0.4,id,title,content,Publisher,Date Published,Financial - Predicted,...,National,Regional,source,country,label,bias,factuality,questionable-source,conspiracy-pseudoscience,pro-science
0,4,4,4,4,prisonplanet--2020-04-13--Migrant Youths Riot ...,Migrant Youths Riot in Belgium Despite Coronav...,Police only intervened when mob was armed.\n\n...,prisonplanet,2020-04-13,0,...,1,0,prisonplanet,USA,1.0,conspiracy-pseudoscience,0.0,0.0,1.0,0.0
1,56,56,56,56,prisonplanet--2020-01-09--Rasmussen Poll: Just...,Rasmussen Poll: Just 21% of Americans Think Je...,Rasmussen Poll: Just 21% of Americans Think Je...,prisonplanet,2020-01-09,0,...,1,0,prisonplanet,USA,1.0,conspiracy-pseudoscience,0.0,0.0,1.0,0.0
2,109,109,109,109,prisonplanet--2020-01-17--Jeffrey Epstein had ...,Jeffrey Epstein had burst capillaries in his e...,Jeffrey Epstein had burst capillaries in his e...,prisonplanet,2020-01-17,0,...,1,0,prisonplanet,USA,1.0,conspiracy-pseudoscience,0.0,0.0,1.0,0.0
3,110,110,110,110,prisonplanet--2020-01-16--‘World Economy Would...,‘World Economy Would Collapse’ If City of Lond...,‘World Economy Would Collapse’ If City of Lond...,prisonplanet,2020-01-16,0,...,1,0,prisonplanet,USA,1.0,conspiracy-pseudoscience,0.0,0.0,1.0,0.0
4,329,329,329,329,prisonplanet--2020-02-17--Woman Who Had Sex Wi...,"Woman Who Had Sex With Pitbull, Hanged Kids Wi...","Woman Who Had Sex With Pitbull, Hanged Kids Wi...",prisonplanet,2020-02-17,0,...,1,0,prisonplanet,USA,1.0,conspiracy-pseudoscience,0.0,0.0,1.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8886,21365,21365,21365,21365,dailyherald--2023-10-17--Jada Pinkett Smith sa...,Jada Pinkett Smith says revealing separation f...,NEW YORK -- Jada Pinkett Smith is feeling free...,dailyherald,2023-10-17,0,...,1,1,dailyherald,USA,0.0,right-center,4.0,0.0,0.0,0.0
8887,21557,21557,21557,21557,dailyherald--2023-11-21--Glendale Heights poli...,"Glendale Heights police sergeant, wife dead in...",A Glendale Heights police sergeant and his wif...,dailyherald,2023-11-21,0,...,1,1,dailyherald,USA,0.0,right-center,4.0,0.0,0.0,0.0
8888,21581,21581,21581,21581,dailyherald--2023-11-23--Thwart the Grinches: ...,Thwart the Grinches: How to keep you and your ...,When you're out shopping for gifts this weeken...,dailyherald,2023-11-23,0,...,1,1,dailyherald,USA,0.0,right-center,4.0,0.0,0.0,0.0
8889,21583,21583,21583,21583,dailyherald--2023-11-24--Thwart the Grinches: ...,Thwart the Grinches: How to keep you and your ...,When you're out shopping for gifts this weeken...,dailyherald,2023-11-24,0,...,1,1,dailyherald,USA,0.0,right-center,4.0,0.0,0.0,0.0


In [163]:
stateinfoadded = pd.merge(regionaldf, countydata, left_on='Publisher', right_on='source', how='inner')
#stateinfoadded.head(15)

In [164]:
us_state_to_abbrev = {
    "Alabama": "AL",
    "Alaska": "AK",
    "Arizona": "AZ",
    "Arkansas": "AR",
    "California": "CA",
    "Colorado": "CO",
    "Connecticut": "CT",
    "Delaware": "DE",
    "Florida": "FL",
    "Georgia": "GA",
    "Hawaii": "HI",
    "Idaho": "ID",
    "Illinois": "IL",
    "Indiana": "IN",
    "Iowa": "IA",
    "Kansas": "KS",
    "Kentucky": "KY",
    "Louisiana": "LA",
    "Maine": "ME",
    "Maryland": "MD",
    "Massachusetts": "MA",
    "Michigan": "MI",
    "Minnesota": "MN",
    "Mississippi": "MS",
    "Missouri": "MO",
    "Montana": "MT",
    "Nebraska": "NE",
    "Nevada": "NV",
    "New Hampshire": "NH",
    "New Jersey": "NJ",
    "New Mexico": "NM",
    "New York": "NY",
    "North Carolina": "NC",
    "North Dakota": "ND",
    "Ohio": "OH",
    "Oklahoma": "OK",
    "Oregon": "OR",
    "Pennsylvania": "PA",
    "Rhode Island": "RI",
    "South Carolina": "SC",
    "South Dakota": "SD",
    "Tennessee": "TN",
    "Texas": "TX",
    "Utah": "UT",
    "Vermont": "VT",
    "Virginia": "VA",
    "Washington": "WA",
    "West Virginia": "WV",
    "Wisconsin": "WI",
    "Wyoming": "WY",
    "District of Columbia": "DC",
    "American Samoa": "AS",
    "Guam": "GU",
    "Northern Mariana Islands": "MP",
    "Puerto Rico": "PR",
    "United States Minor Outlying Islands": "UM",
    "U.S. Virgin Islands": "VI",
}
    
# invert the dictionary
abbrev_to_us_state = dict(map(reversed, us_state_to_abbrev.items()))

In [165]:
stateinfoadded.to_csv('stateinfoadded.csv')

In [166]:
len(stateinfoadded['county'].unique())

157

In [167]:
mhpsas = pd.read_csv('mhpsa.csv')
regionalonlyhealth = stateinfoadded

In [168]:
def func(row):
    val = row['state_abbr']
    return abbrev_to_us_state[val]
    
mhpsas['State'] = mhpsas.apply(func, axis=1)

In [169]:
onlyregionainfo = regionalonlyhealth.groupby(['county', 'state', HealthPredicted]).size().reset_index(name='Count')
onlyregionainfo = onlyregionainfo.sort_values(by='Count', ascending = False)
onlyregionainfo

,county,state,Health - Predicted,Count
132,King,Washington,0,314
147,Los Angeles,California,0,266
62,Cook,Illinois,0,188
19,Bernalillo,New Mexico,0,150
148,Los Angeles,California,1,133
...,...,...,...,...
240,Scotland,North Carolina,0,1
146,Livingston,Missouri,0,1
1,Acadia,Louisiana,1,1
151,Marion,Oregon,0,1


In [170]:
maindfcounty = onlyregionainfo['county'].to_list()
maindfstate = onlyregionainfo['state'].to_list()
maindffreq = onlyregionainfo['Count'].to_list()

shortageareascounty = mhpsas['county_name'].to_list()
shortageareasstate = mhpsas['State'].to_list()
shortageareasvalue = mhpsas['value'].to_list()

In [171]:
print(len(set(maindfstate)))

45


In [172]:
founds = 0
foundcountyisshortagearea = 0
articlesnotinshortagearea = 0
totalarticles = len(regionalonlyhealth)

for i in range(len(maindfcounty)):
    present = False
    currcounty = maindfcounty[i]
    currstate = maindfstate[i]
    for j in range(len(shortageareascounty)):
        if currcounty in shortageareascounty[j] and currstate == shortageareasstate[j]:
            print(f"Element: {shortageareascounty[j]} {shortageareasstate[j]} {currcounty} {currstate}, Frequency: {maindffreq[i]}")
            present = True
            founds = founds + 1
            if shortageareasvalue[j] not in "None of county is shortage area":
                foundcountyisshortagearea = foundcountyisshortagearea + 1
            else:
                print("Not a shortage area")
                articlesnotinshortagearea = articlesnotinshortagearea + maindffreq[i]
            break
    if (present == False):
        print(f"NOT FOUND: {currcounty} {currstate}, Frequency: {maindffreq[i]}")
            

print("Matched counties: ",founds)
print("Total number of counties in our articles: ",len(maindfcounty))
print("Counties in our articles that are shortage areas",foundcountyisshortagearea)
print("Total number of articles: ", totalarticles)
print("Total number of articles NOT in shortage areas: ", articlesnotinshortagearea)

Element: King County Washington King Washington, Frequency: 314
Element: Los Angeles County California Los Angeles California, Frequency: 266
Element: Cook County Illinois Cook Illinois, Frequency: 188
Element: Bernalillo County New Mexico Bernalillo New Mexico, Frequency: 150
Element: Los Angeles County California Los Angeles California, Frequency: 133
Element: Kennebec County Maine Kennebec Maine, Frequency: 132
Element: Clark County Washington Clark Washington, Frequency: 129
Element: Clark County Nevada Clark Nevada, Frequency: 97
Element: King County Washington King Washington, Frequency: 95
Element: Suffolk County Massachusetts Suffolk Massachusetts, Frequency: 95
Element: Denver County Colorado Denver Colorado, Frequency: 94
Element: Santa Clara County California Santa Clara California, Frequency: 73
Element: Cook County Illinois Cook Illinois, Frequency: 73
Element: Allegheny County Pennsylvania Allegheny Pennsylvania, Frequency: 70
Not a shortage area
Element: Westmoreland Cou

In [173]:
print("Left bias: ", leftbias.shape[0])
print("Financial %: ", 100 * leftbias[leftbias[FinancialPredicted] > 0].shape[0]/leftbias.shape[0])
print("Legal %: ", 100 * leftbias[leftbias[LegalPredicted] > 0].shape[0]/leftbias.shape[0])
print("School %: ", 100 * leftbias[leftbias[SchoolPredicted] > 0].shape[0]/leftbias.shape[0])
print("Health %: ", 100 * leftbias[leftbias[HealthPredicted] > 0].shape[0]/leftbias.shape[0])
print("Not any category %: ", 100 * leftbias[leftbias['None - Predicted'] > 0].shape[0]/leftbias.shape[0])

Left bias:  5651
Financial %:  5.308794903556892
Legal %:  35.09113431251106
School %:  5.662714563794019
Health %:  26.03079101044063
Not any category %:  32.64908865687489


In [174]:
print("Left bias: ", leftbias.shape[0])
print("Financial: ", leftbias[leftbias[FinancialPredicted] > 0].shape[0])
print("Legal: ", leftbias[leftbias[LegalPredicted] > 0].shape[0])
print("School: ", leftbias[leftbias[SchoolPredicted] > 0].shape[0])
print("Health: ", leftbias[leftbias[HealthPredicted] > 0].shape[0])
print("Not any category: ", leftbias[leftbias['None - Predicted'] > 0].shape[0])

Left bias:  5651
Financial:  300
Legal:  1983
School:  320
Health:  1471
Not any category:  1845


In [175]:
print("Right bias: ", rightbias.shape[0])
print("Financial %: ", rightbias[rightbias[FinancialPredicted] > 0].shape[0])
print("Legal %: ", rightbias[rightbias[LegalPredicted] > 0].shape[0])
print("School %: ", rightbias[rightbias[SchoolPredicted] > 0].shape[0])
print("Health %: ", rightbias[rightbias[HealthPredicted] > 0].shape[0])
print("Not any category %: ", rightbias[rightbias['None - Predicted'] > 0].shape[0])

Right bias:  1811
Financial %:  123
Legal %:  508
School %:  128
Health %:  501
Not any category %:  633


In [176]:
print("Right bias: ", rightbias.shape[0])
print("Financial %: ", 100 * rightbias[rightbias[FinancialPredicted] > 0].shape[0]/rightbias.shape[0])
print("Legal %: ", 100 * rightbias[rightbias[LegalPredicted] > 0].shape[0]/rightbias.shape[0])
print("School %: ", 100 * rightbias[rightbias[SchoolPredicted] > 0].shape[0]/rightbias.shape[0])
print("Health %: ", 100 * rightbias[rightbias[HealthPredicted] > 0].shape[0]/rightbias.shape[0])
print("Not any category %: ", 100 * rightbias[rightbias['None - Predicted'] > 0].shape[0]/rightbias.shape[0])

Right bias:  1811
Financial %:  6.791827719491994
Legal %:  28.050800662617338
School %:  7.06791827719492
Health %:  27.664273881833243
Not any category %:  34.953064605190505


In [177]:
us_states = [
    'Alabama', 'Alaska', 'Arizona', 'Arkansas', 'California', 'Colorado', 'Connecticut', 'Delaware',
    'Florida', 'Georgia', 'Hawaii', 'Idaho', 'Illinois', 'Indiana', 'Iowa', 'Kansas', 'Kentucky',
    'Louisiana', 'Maine', 'Maryland', 'Massachusetts', 'Michigan', 'Minnesota', 'Mississippi', 'Missouri',
    'Montana', 'Nebraska', 'Nevada', 'New Hampshire', 'New Jersey', 'New Mexico', 'New York',
    'North Carolina', 'North Dakota', 'Ohio', 'Oklahoma', 'Oregon', 'Pennsylvania', 'Rhode Island',
    'South Carolina', 'South Dakota', 'Tennessee', 'Texas', 'Utah', 'Vermont', 'Virginia', 'Washington',
    'West Virginia', 'Wisconsin', 'Wyoming'
]

In [178]:
# import geopandas as gpd
# import pandas as pd
# import matplotlib.pyplot as plt

# def plotheatmpus(label = "", title = ""):
#     states = gpd.read_file("/Users/minijain/Downloads/cb_2018_us_state_20m/cb_2018_us_state_20m.shp")
#     merged = states.merge(statesinfos, left_on='NAME', right_on='State')
#     fig, ax = plt.subplots(figsize=(6,6))
#     ax = merged.plot(column=label, cmap="Reds", linewidth=0.4, ax=ax, edgecolor=".4", legend=True, legend_kwds={ 'shrink': 0.3},)

#     ax.set_xlim(-170, -64)
#     ax.set_ylim(25, 72)

#     ax.axis("off")
#     plt.title(title, loc='center')
#     plt.show()

In [179]:
# for k in keys:
#     plotheatmpus(k, 'Articles with ' + k + ' as a cause of suicide')

In [180]:
from numpy import var, mean, sqrt
import math

regionalornot = np.array(result['Regional'].to_list())
nationalornot = np.array(result['National'].to_list())
leftornot = np.array(result['Left'].to_list())
rightornot = np.array(result['Right'].to_list())

finanicalproblem = np.array(result[FinancialPredicted].to_list())
legalproblem = np.array(result[LegalPredicted].to_list())
schoolproblem = np.array(result[SchoolPredicted].to_list())
healthproblem = np.array(result[HealthPredicted].to_list())

def cohend(d1, d2):
    # calculate the size of samples
    n1, n2 = len(d1), len(d2)
    # calculate the variance of the samples
    s1, s2 = var(d1, ddof=1), var(d2, ddof=1)
    # calculate the pooled standard deviation
    s = sqrt(((n1 - 1) * s1 + (n2 - 1) * s2) / (n1 + n2 - 2))
    # calculate the means of the samples
    u1, u2 = mean(d1), mean(d2)
    # calculate the effect size
    return (u1 - u2) / s


### Unpaired t test

In [181]:

# null hypothesis : No difference between the distribution of Financial articles across left and right leaning articles
# reject the null hypothesis is p-value < 0.05
financialleft = finanicalproblem[np.argwhere(leftornot > 0).flatten()]
financialright = finanicalproblem[np.argwhere(rightornot > 0).flatten()]
financialregional = finanicalproblem[np.argwhere(regionalornot > 0).flatten()]
financialnational = finanicalproblem[np.argwhere(nationalornot > 0).flatten()]

print(FinancialLabel)

from scipy import stats
#print(stats.ttest_ind(financialleft, financialright))
ttest = stats.ttest_ind(financialright, financialleft, equal_var=False)
print("Political p-value: ", "{:.10f}".format(ttest.pvalue))
if (ttest.pvalue < 0.05):
    print("Distributions different")
else:
    print("Distributions same")
print("cohen: ", cohend(financialright, financialleft))

ttest = stats.ttest_ind(financialregional, financialnational, equal_var=False)
print("Regionality p-value: ", "{:.10f}".format(ttest.pvalue))
if (ttest.pvalue < 0.05):
    print("Distributions different")
else:
    print("Distributions same")
print("cohen: ", cohend(financialregional, financialnational))

Financial/Job Problem
Political p-value:  0.0252344942
Distributions different
cohen:  0.06414847637735692
Regionality p-value:  0.4364165152
Distributions same
cohen:  0.014445999979570306


In [182]:
# null hypothesis : No difference between the distribution of Legal articles across left and right leaning articles
# reject the null hypothesis is p-value < 0.05

legalleft = legalproblem[np.argwhere(leftornot > 0).flatten()]
legalright = legalproblem[np.argwhere(rightornot > 0).flatten()]
legalregional = legalproblem[np.argwhere(regionalornot > 0).flatten()]
legalnational = legalproblem[np.argwhere(nationalornot > 0).flatten()]

from scipy import stats
print(Legallabel)
#print(stats.ttest_ind(legalleft, legalright))
ttest = stats.ttest_ind(legalleft, legalright, equal_var=False)
print("Political p-value: ", "{:.20f}".format(ttest.pvalue))
if (ttest.pvalue < 0.05):
    print("Distributions different")
else:
    print("Distributions same")
print("cohen: ", cohend(legalleft, legalright))

ttest = stats.ttest_ind(legalregional, legalnational, equal_var=False)
print("Regionality p-value: ", "{:.20f}".format(ttest.pvalue))
if (ttest.pvalue < 0.05):
    print("Distributions different")
else:
    print("Distributions same")
print("cohen: ", cohend(legalregional, legalnational))


Legal Problem
Political p-value:  0.00000001204863893118
Distributions different
cohen:  0.14957978421124263
Regionality p-value:  0.00000000516635418356
Distributions different
cohen:  -0.10611349542568964


In [183]:
# null hypothesis : No difference between the distribution of School articles across left and right leaning articles
# reject the null hypothesis is p-value < 0.05

schoolleft = schoolproblem[np.argwhere(leftornot > 0).flatten()]
schoolright = schoolproblem[np.argwhere(rightornot > 0).flatten()]
schoolnational = schoolproblem[np.argwhere(nationalornot > 0).flatten()]
schoolregional = schoolproblem[np.argwhere(regionalornot > 0).flatten()]
print(SchoolLabel)
from scipy import stats
#print(stats.ttest_ind(schoolleft, schoolright))
ttest = stats.ttest_ind(schoolleft, schoolright, equal_var=False)
print("Political p-value: ", "{:.10f}".format(ttest.pvalue))
if (ttest.pvalue < 0.05):
    print("Distributions different")
else:
    print("Distributions same")
print("cohen: ", cohend(schoolleft, schoolright))

ttest = stats.ttest_ind(schoolnational, schoolregional, equal_var=False)
print("Regionality p-value: ", "{:.10f}".format(ttest.pvalue))
if (ttest.pvalue < 0.05):
    print("Distributions different")
else:
    print("Distributions same")
print("cohen: ", cohend(schoolnational, schoolregional))

School or Academic Related Problem
Political p-value:  0.0378326730
Distributions different
cohen:  -0.059163536961422195
Regionality p-value:  0.2942699047
Distributions same
cohen:  -0.019525598314074133


In [184]:
# null hypothesis : No difference between the distribution of Health articles across left and right leaning articles
# reject the null hypothesis is p-value < 0.05

healthleft = healthproblem[np.argwhere(leftornot > 0).flatten()]
healthright = healthproblem[np.argwhere(rightornot > 0).flatten()]
healthnational = healthproblem[np.argwhere(nationalornot > 0).flatten()]
healthregional = healthproblem[np.argwhere(regionalornot > 0).flatten()]
print(HealthLabel)
from scipy import stats
#print(stats.ttest_ind(healthleft, healthright))
ttest = stats.ttest_ind(healthleft, healthright, equal_var=False)
print("Political p-value: ", "{:.10f}".format(ttest.pvalue))
if (ttest.pvalue < 0.05):
    print("Distributions different")
else:
    print("Distributions same")
print("cohen: ", cohend(healthleft, healthright))

ttest = stats.ttest_ind(healthnational, healthregional, equal_var=False)
print("Regionality p-value: ", "{:.20f}".format(ttest.pvalue))
if (ttest.pvalue < 0.05):
    print("Distributions different")
else:
    print("Distributions same")
print("cohen: ", cohend(healthnational, healthregional))

Lack of Access to Health/Mental Health Care
Political p-value:  0.1744927138
Distributions same
cohen:  -0.03704475876263217
Regionality p-value:  0.00000000000000000390
Distributions different
cohen:  -0.16413318129073498


### Chi Square Test

In [185]:
import numpy as np
from scipy.stats import chi2_contingency

# Null hypothesis: The probability of article being financial in case of left leaning is equivalent to that of right leaning.
# Reject null hypothesis if p value < 0.05

# Create a contingency table
# 300  109 
# 5381 1472
print(FinancialLabel)
# Financial Problem
contingency_table = np.array([[300, 109], [5381, 1472]])

chi2, p, dof, expected = chi2_contingency(contingency_table)

print("Political p-value:", "{:.10f}".format(p))
if (p < 0.05):
    print("Distributions different")
else:
    print("Distributions same")

contingency_table = np.array([[506, 239], [8148, 3813]])

chi2, p, dof, expected = chi2_contingency(contingency_table)

print("Regionality p-value:", "{:.10f}".format(p))
if (p < 0.05):
    print("Distributions different")
else:
    print("Distributions same")

Financial/Job Problem
Political p-value: 0.0164018682
Distributions different
Regionality p-value: 0.9408259867
Distributions same


In [186]:
import numpy as np
from scipy.stats import chi2_contingency

# Null hypothesis: The probability of article being legal in case of left leaning is equivalent to that of right leaning.
# Reject null hypothesis if p value < 0.05

# Create a contingency table
# 300  109 
# 5381 1472
print(Legallabel)
# Legal Problem
#contingency_table = np.array([[1983, 416], [3698, 1165]])
contingency_table = np.array([[1983, 416], [3698, 1165]])

chi2, p, dof, expected = chi2_contingency(contingency_table)

print("Political p-value:", "{:.10f}".format(p))
if (p < 0.05):
    print("Distributions different")
else:
    print("Distributions same")

contingency_table = np.array([[2994, 1218], [5660, 2834]])

chi2, p, dof, expected = chi2_contingency(contingency_table)

print("Regionality p-value:", "{:.10f}".format(p))
if (p < 0.05):
    print("Distributions different")
else:
    print("Distributions same")

Legal Problem
Political p-value: 0.0000000002
Distributions different
Regionality p-value: 0.0000004573
Distributions different


In [187]:
import numpy as np
from scipy.stats import chi2_contingency

# Null hypothesis: The probability of article being school in case of left leaning is equivalent to that of right leaning.
# Reject null hypothesis if p value < 0.05

# Create a contingency table
# 300  109 
# 5381 1472
print(SchoolLabel)
# School Problem
#contingency_table = np.array([[1983, 416], [3698, 1165]])
contingency_table = np.array([[320, 115], [5361, 1466]])

chi2, p, dof, expected = chi2_contingency(contingency_table)

print("Political p-value:", "{:.10f}".format(p))
if (p < 0.05):
    print("Distributions different")
else:
    print("Distributions same")

contingency_table = np.array([[514, 264], [8140, 3788]])

chi2, p, dof, expected = chi2_contingency(contingency_table)

print("Regionality p-value:", "{:.10f}".format(p))
if (p < 0.05):
    print("Distributions different")
else:
    print("Distributions same")

School or Academic Related Problem
Political p-value: 0.0176853767
Distributions different
Regionality p-value: 0.2216759389
Distributions same


In [188]:
import numpy as np
from scipy.stats import chi2_contingency

# Null hypothesis: The probability of article being health in case of left leaning is equivalent to that of right leaning.
# Reject null hypothesis if p value < 0.05

# Create a contingency table
# 300  109 
# 5381 1472
print(HealthLabel)
# Health Problem
#contingency_table = np.array([[1983, 416], [3698, 1165]])
contingency_table = np.array([[1379, 395], [4302, 1186]])

chi2, p, dof, expected = chi2_contingency(contingency_table)

print("Political p-value:", "{:.10f}".format(p))
if (p < 0.05):
    print("Distributions different")
else:
    print("Distributions same")
contingency_table = np.array([[2038, 1104], [6616, 2948]])

chi2, p, dof, expected = chi2_contingency(contingency_table)

print("Regionality p-value:", "{:.10f}".format(p))
if (p < 0.05):
    print("Distributions different")
else:
    print("Distributions same")

Lack of Access to Health/Mental Health Care
Political p-value: 0.5834932005
Distributions same
Regionality p-value: 0.0000075205
Distributions different


In [291]:
import liwc
from nltk.tokenize import word_tokenize
from collections import Counter
parse, category_names = liwc.load_token_parser('/Users/minijain/Downloads/SocWebLab/Structural_Violence/LIWC_chopping_board/LIWC2015_English_Flat.dic')

def liwcfunc(sentence_to_parse):
    tokenized_sentence = word_tokenize(sentence_to_parse)
    
    liwc_counts = Counter(category for token in tokenized_sentence for category in parse(token))
    total_words = len(tokenized_sentence)
    normalized_liwc_counts = {category: 100 * (count / total_words) for category, count in liwc_counts.items()}
    #print(normalized_liwc_counts)
    return normalized_liwc_counts

sentence_to_parse = "Even in lockdown , Tom Jones is living his best life .\nThe man who swiveled his hips into millions of homes via a hit television variety show in the late ’ 60s and early ’ 70s , palled around with Elvis and Sinatra , and enjoyed a remarkable third act in his career via scene-stealing turns in The Fresh Prince of Bel Air and Mars Attacks !\n—not to mention a global smash with a cover of Prince ’ s “ Kiss ” in 1994—is now in the midst of a remarkable late-career renaissance .\nWith a fantastic new @ @ @ @ @ @ @ now—the fourth in a string of excellent and eclectic releases over the past 10 years helmed by producer Ethan Johns—and fully vaccinated , the proud Welshman is chomping at the bit to get back out on the live stage that made him a household name .\nIn a recent Zoom interview with The Daily Beast , Sir Tom ( he was knighted in 2005 ) reflects on his long career and remembers his pals Elvis Presley and Frank Sinatra , his run-ins with Donald Trump , and why he wasn ’ t wrong @ @ @ @ @ @ @ makes a plea : “ Everyone has got to get the jab !\n” The ages in my family run from 94 down to 18 and , independently of each other , everyone knows and loves you .\nFrom your early hits and your variety show , to the Fresh Prince on down to Spotify playlists , you enjoy a weird sort of omnipresence .\nWell , it ’ s versatility , I think .\nThe idea of appealing to a wide variety of people , that started when I was a boy @ @ @ @ @ @ @ of cousins , so there were a lot of weddings that I used to go to and sing at .\nIf we were at a wedding , or any gathering , I would pull on my mother ’ s skirt and say , “ Mom , when are they going to ask me to sing ?\n” I was always ready .\nWhen I was very young , I loved Al Jolson .\nI went to see The Jolson Story , and that was a big influence .\nThe problem , of course @ @ @ @ @ @ @ but he had a great voice .\nAnd I loved cowboy songs .\nThere was a guy named Vaughn Monroe who had a song called “ ( Ghost ) Riders in the Sky.\n” Oh , that was a big "
liwcfunc(sentence_to_parse)


{'function': 34.375,
 'prep': 12.276785714285714,
 'relativ': 11.830357142857142,
 'space': 7.142857142857142,
 'auxverb': 3.571428571428571,
 'verb': 9.375,
 'focuspresent': 4.464285714285714,
 'bio': 1.5625,
 'health': 0.6696428571428571,
 'pronoun': 6.4732142857142865,
 'ppron': 4.241071428571429,
 'shehe': 2.232142857142857,
 'social': 9.151785714285714,
 'male': 3.3482142857142856,
 'adj': 3.7946428571428568,
 'compare': 1.1160714285714286,
 'affect': 3.125,
 'posemo': 2.455357142857143,
 'drives': 5.580357142857143,
 'achiev': 0.6696428571428571,
 'power': 1.5625,
 'reward': 1.1160714285714286,
 'ipron': 2.232142857142857,
 'interrog': 1.1160714285714286,
 'motion': 1.5625,
 'body': 0.2232142857142857,
 'number': 0.6696428571428571,
 'home': 0.6696428571428571,
 'article': 6.4732142857142865,
 'leisure': 1.3392857142857142,
 'quant': 1.5625,
 'percept': 2.455357142857143,
 'see': 0.6696428571428571,
 'time': 3.3482142857142856,
 'conj': 4.241071428571429,
 'adverb': 2.23214285714

In [292]:
newcats = category_names.copy()
newcats.insert(0, 'id')
liwcanalysis = pd.DataFrame(columns=newcats)
liwcanalysis

,id,function,pronoun,ppron,i,we,you,shehe,they,ipron,...,home,money,relig,death,informal,swear,netspeak,assent,nonflu,filler


In [293]:
liwcanalysis['id'] = result['id']
liwcanalysis.fillna(0.0, inplace=True)

In [294]:
liwc_results = result['content'].apply(liwcfunc)

In [295]:
liwc_results

4        {'function': 36.97916666666667, 'adverb': 2.60...
7        {'function': 36.869474778991155, 'prep': 11.56...
9        {'function': 27.647058823529413, 'prep': 12.94...
12       {'function': 46.32768361581921, 'adverb': 5.64...
17       {'function': 30.952380952380953, 'conj': 2.827...
                               ...                        
22013    {'social': 5.726872246696035, 'verb': 9.515418...
22016    {'relativ': 12.549800796812749, 'time': 3.1872...
22017    {'social': 12.616822429906541, 'male': 4.90654...
22018    {'social': 12.616822429906541, 'male': 4.90654...
22020    {'relativ': 12.5, 'time': 3.1746031746031744, ...
Name: content, Length: 13629, dtype: object

In [296]:
inds = result.index.to_list()

In [297]:
liwcanalysis

,id,function,pronoun,ppron,i,we,you,shehe,they,ipron,...,home,money,relig,death,informal,swear,netspeak,assent,nonflu,filler
4,prisonplanet--2020-04-13--Migrant Youths Riot ...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
7,theintercept--2020-04-12--“Burials Are Cheaper...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
9,"breaking911--2020-01-06--Handwritten Note, Pho...",0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
12,dailybeast--2020-01-07--DOJ’s New Argument: Tr...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
17,washingtonpost--2020-01-06--A GOP congressman ...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
22013,dailybreeze--2024-02-12--South Bay history: Ho...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
22016,thekennebecjournal--2024-02-12--A shooter open...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
22017,bainbridgeislandreview--2024-02-12--Man assaul...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
22018,centralkitsapreporter--2024-02-12--Man assault...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [298]:
c = 0
for i in liwc_results:
    for key, value in i.items():
        liwcanalysis.at[inds[c], key] = value
        #print(liwcanalysis)
    c = c+1

In [299]:
liwcanalysis['Publisher'] = result['Publisher']

In [300]:
nationalindex = nationaldf[nationaldf[FinancialPredicted] + nationaldf[LegalPredicted] + nationaldf[SchoolPredicted] + nationaldf[HealthPredicted] > 0].index.to_list()
regionalindex = regionaldf[regionaldf[FinancialPredicted] + regionaldf[LegalPredicted] + regionaldf[SchoolPredicted] + regionaldf[HealthPredicted] > 0].index.to_list()
leftindex = leftbias[leftbias[FinancialPredicted] + leftbias[LegalPredicted] + leftbias[SchoolPredicted] + leftbias[HealthPredicted] > 0].index.to_list()
rightindex = rightbias[rightbias[FinancialPredicted] + rightbias[LegalPredicted] + rightbias[SchoolPredicted] + rightbias[HealthPredicted] > 0].index.to_list()

In [301]:
ind = result[result[FinancialPredicted] + result[LegalPredicted] + result[SchoolPredicted] + result[HealthPredicted] > 0].index.to_list()
liwcanalysis = liwcanalysis[liwcanalysis.index.isin(ind)]

In [302]:
liwcanalysis

,id,function,pronoun,ppron,i,we,you,shehe,they,ipron,...,money,relig,death,informal,swear,netspeak,assent,nonflu,filler,Publisher
4,prisonplanet--2020-04-13--Migrant Youths Riot ...,36.979167,6.510417,3.385417,1.041667,0.520833,0.260417,0.520833,1.041667,3.125000,...,1.041667,0.000000,0.520833,0.000000,0.000000,0.0,0.00000,0.000000,0.0,prisonplanet
7,theintercept--2020-04-12--“Burials Are Cheaper...,36.869475,7.869648,3.900156,0.260010,0.450685,0.312012,1.473392,1.404056,3.952158,...,0.520021,0.034668,0.260010,0.086670,0.034668,0.0,0.00000,0.052002,0.0,theintercept
9,"breaking911--2020-01-06--Handwritten Note, Pho...",27.647059,3.529412,3.529412,1.764706,0.000000,0.000000,1.764706,0.000000,0.000000,...,0.000000,0.000000,4.117647,0.000000,0.000000,0.0,0.00000,0.000000,0.0,breaking911
17,washingtonpost--2020-01-06--A GOP congressman ...,30.952381,4.464286,1.488095,0.000000,0.148810,0.148810,1.041667,0.148810,2.976190,...,0.297619,0.000000,0.744048,0.148810,0.000000,0.0,0.14881,0.000000,0.0,washingtonpost
21,veteranstoday--2020-01-07--Trump’s Shameless M...,39.063304,8.852290,4.632012,0.308801,0.926402,0.360268,1.492537,1.544004,4.220278,...,0.360268,0.720535,1.595471,0.102934,0.051467,0.0,0.00000,0.051467,0.0,veteranstoday
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
22007,nevadabusinessjournal--2024-02-08--Avery Burto...,25.454545,1.616162,1.010101,0.000000,0.202020,0.000000,0.808081,0.000000,0.606061,...,0.000000,0.202020,0.000000,0.000000,0.000000,0.0,0.00000,0.000000,0.0,nevadabusinessjournal
22011,thesuffolktimes--2024-02-11--Editorial: From c...,32.878581,7.776262,3.819918,0.000000,1.500682,0.272851,0.682128,1.364256,3.956344,...,0.545703,0.000000,0.409277,0.000000,0.000000,0.0,0.00000,0.000000,0.0,thesuffolktimes
22012,chicagosuntimes--2024-02-12--'The Dynasty' rev...,33.039648,5.726872,3.083700,0.000000,0.550661,0.550661,1.651982,0.330396,2.643172,...,0.330396,0.000000,0.220264,0.330396,0.000000,0.0,0.00000,0.330396,0.0,chicagosun-times
22017,bainbridgeislandreview--2024-02-12--Man assaul...,32.710280,5.841121,5.140187,0.000000,0.000000,0.000000,4.205607,0.934579,0.700935,...,0.233645,0.000000,1.168224,0.000000,0.000000,0.0,0.00000,0.000000,0.0,bainbridgeislandreview


In [303]:
grouped = liwcanalysis.drop(columns=['id']).groupby('Publisher')
fin = grouped.mean()
fin

,function,pronoun,ppron,i,we,you,shehe,they,ipron,article,...,home,money,relig,death,informal,swear,netspeak,assent,nonflu,filler
Publisher,,,,,,,,,,,,,,,,,,,,,
21stcenturywire,35.478845,6.871881,3.156111,0.000000,0.103011,0.070802,2.238952,0.743346,3.715770,5.918810,...,0.073512,1.270098,0.000000,1.613227,0.108913,0.000000,0.044170,0.029343,0.035401,0.000000
abcnews,33.481963,5.937099,3.453834,0.059288,0.102902,0.056270,2.626907,0.608466,2.482683,6.550845,...,0.540575,0.673534,0.079726,1.195688,0.048253,0.008433,0.007399,0.009734,0.021674,0.000662
advertiserdemocrat,35.801937,7.305427,3.888671,0.166996,0.186389,0.252258,2.232487,1.050540,3.416757,5.729389,...,0.549938,0.481834,0.024938,0.399123,0.110629,0.012912,0.000000,0.046925,0.050792,0.000000
alamedatimesstar,34.524475,7.202750,4.650034,0.222818,0.094192,0.171065,3.693305,0.468655,2.552716,6.282650,...,0.693010,0.571286,0.077815,1.903093,0.046936,0.000000,0.008352,0.002784,0.035800,0.000000
albertleatribune,38.898663,8.855071,4.344177,0.211847,0.378496,0.172812,2.950309,0.630714,4.498220,6.092649,...,0.912340,0.600922,0.066597,0.276507,0.336117,0.101394,0.012674,0.025349,0.196700,0.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
wilmingtonnewsjournal,29.134179,4.044798,2.334035,0.063861,0.146038,0.084790,1.618581,0.420765,1.710763,6.677452,...,0.288527,0.387338,0.072369,0.986343,0.058751,0.008989,0.006133,0.006299,0.039633,0.000000
wisecountymessenger,37.708649,9.635812,5.918058,0.151745,0.227618,0.606980,4.628225,0.303490,3.717754,6.600910,...,0.227618,0.303490,0.227618,0.379363,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
x22report,32.220677,5.053315,1.900788,0.092721,0.092721,0.417246,0.463607,0.834492,3.152527,6.490496,...,0.463607,1.854427,0.000000,0.139082,0.046361,0.046361,0.000000,0.000000,0.000000,0.000000


In [304]:
grouped = fin.T
grouped.columns = ['' if col == 'Publisher' else col for col in grouped.columns]
grouped

,21stcenturywire,abcnews,advertiserdemocrat,alamedatimesstar,albertleatribune,albuquerquejournal,andalusiastarnews,arizonacapitoltimes,arkansastimes,atlantadailyworld,...,waterburyrepublicanamerican,westcentraltribune,westernjournal,westhawaiitoday,whidbeynewstimes,wilmingtonnewsjournal,wisecountymessenger,x22report,yahoonews,yellowspringsnews
function,35.478845,33.481963,35.801937,34.524475,38.898663,35.302356,36.010738,35.807091,36.444183,38.112088,...,35.263611,32.772460,34.986627,34.370626,34.303386,29.134179,37.708649,32.220677,33.698753,40.686586
pronoun,6.871881,5.937099,7.305427,7.202750,8.855071,6.359127,6.825830,5.985869,6.199100,8.203610,...,6.148336,5.263193,7.181060,5.717070,4.732050,4.044798,9.635812,5.053315,6.141577,13.477432
ppron,3.156111,3.453834,3.888671,4.650034,4.344177,3.628364,3.811481,3.351487,2.982150,5.001930,...,3.387017,3.185462,4.024918,2.737553,2.557960,2.334035,5.918058,1.900788,3.317295,8.264463
i,0.000000,0.059288,0.166996,0.222818,0.211847,0.118590,0.127049,0.099800,0.180087,0.307094,...,0.062194,0.034825,0.172652,0.110485,0.000000,0.063861,0.151745,0.092721,0.231489,1.716465
we,0.103011,0.102902,0.186389,0.094192,0.378496,0.175750,0.795742,1.466303,0.435371,1.541187,...,0.151182,0.265900,0.439710,0.227992,0.000000,0.146038,0.227618,0.092721,0.241832,0.699301
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
swear,0.000000,0.008433,0.012912,0.000000,0.101394,0.011295,0.000000,0.000000,0.007627,0.000000,...,0.003773,0.000000,0.010993,0.002803,0.000000,0.008989,0.000000,0.046361,0.017596,0.063573
netspeak,0.044170,0.007399,0.000000,0.008352,0.012674,0.012081,0.044131,0.000000,0.000000,0.066007,...,0.020667,0.000000,0.006366,0.001586,0.000000,0.006133,0.000000,0.000000,0.025662,0.000000
assent,0.029343,0.009734,0.046925,0.002784,0.025349,0.009503,0.000000,0.038625,0.033044,0.000000,...,0.011901,0.000000,0.022654,0.011748,0.000000,0.006299,0.000000,0.000000,0.014888,0.063573
nonflu,0.035401,0.021674,0.050792,0.035800,0.196700,0.032086,0.000000,0.000000,0.073686,0.176832,...,0.040074,0.025735,0.040936,0.008572,0.022745,0.039633,0.000000,0.000000,0.043386,0.127146


In [305]:
grouped.to_csv('liwc scores by publisher.csv')

In [306]:
df_new = liwcanalysis.drop(columns=['id', 'Publisher'])

In [307]:
df_new

,function,pronoun,ppron,i,we,you,shehe,they,ipron,article,...,home,money,relig,death,informal,swear,netspeak,assent,nonflu,filler
4,36.979167,6.510417,3.385417,1.041667,0.520833,0.260417,0.520833,1.041667,3.125000,7.031250,...,0.260417,1.041667,0.000000,0.520833,0.000000,0.000000,0.0,0.00000,0.000000,0.0
7,36.869475,7.869648,3.900156,0.260010,0.450685,0.312012,1.473392,1.404056,3.952158,6.673600,...,0.312012,0.520021,0.034668,0.260010,0.086670,0.034668,0.0,0.00000,0.052002,0.0
9,27.647059,3.529412,3.529412,1.764706,0.000000,0.000000,1.764706,0.000000,0.000000,3.529412,...,0.588235,0.000000,0.000000,4.117647,0.000000,0.000000,0.0,0.00000,0.000000,0.0
17,30.952381,4.464286,1.488095,0.000000,0.148810,0.148810,1.041667,0.148810,2.976190,6.994048,...,0.000000,0.297619,0.000000,0.744048,0.148810,0.000000,0.0,0.14881,0.000000,0.0
21,39.063304,8.852290,4.632012,0.308801,0.926402,0.360268,1.492537,1.544004,4.220278,6.330417,...,0.051467,0.360268,0.720535,1.595471,0.102934,0.051467,0.0,0.00000,0.051467,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
22007,25.454545,1.616162,1.010101,0.000000,0.202020,0.000000,0.808081,0.000000,0.606061,5.656566,...,0.606061,0.000000,0.202020,0.000000,0.000000,0.000000,0.0,0.00000,0.000000,0.0
22011,32.878581,7.776262,3.819918,0.000000,1.500682,0.272851,0.682128,1.364256,3.956344,4.229195,...,0.409277,0.545703,0.000000,0.409277,0.000000,0.000000,0.0,0.00000,0.000000,0.0
22012,33.039648,5.726872,3.083700,0.000000,0.550661,0.550661,1.651982,0.330396,2.643172,5.947137,...,0.220264,0.330396,0.000000,0.220264,0.330396,0.000000,0.0,0.00000,0.330396,0.0
22017,32.710280,5.841121,5.140187,0.000000,0.000000,0.000000,4.205607,0.934579,0.700935,4.906542,...,0.467290,0.233645,0.000000,1.168224,0.000000,0.000000,0.0,0.00000,0.000000,0.0


In [308]:
temp = df_new[df_new.index.isin(nationalindex)]
nationalmean = temp.mean()
temp = df_new[df_new.index.isin(regionalindex)]
regionalmean = temp.mean()
temp = df_new[df_new.index.isin(leftindex)]
leftmean = temp.mean()
temp = df_new[df_new.index.isin(rightindex)]
rightmean = temp.mean()

In [309]:
finaldf = pd.DataFrame()
finaldf['Category'] = rightmean.index
finaldf['National'] = nationalmean.to_list()
finaldf['Regional'] = regionalmean.to_list()
finaldf['Left'] = leftmean.to_list()
finaldf['Right'] = rightmean.to_list()

In [310]:
finaldf.to_csv('liwc means.csv')

In [311]:
finaldf

,Category,National,Regional,Left,Right
0,function,34.440591,34.771790,33.849244,34.380771
1,pronoun,6.560469,6.405362,6.415033,6.625656
2,ppron,3.691957,3.610872,3.670871,3.696225
3,i,0.187022,0.177641,0.162504,0.164509
4,we,0.248523,0.245572,0.216771,0.274446
...,...,...,...,...,...
68,swear,0.016949,0.010196,0.015130,0.018505
69,netspeak,0.014138,0.009354,0.011205,0.010412
70,assent,0.015390,0.013406,0.014422,0.012421
71,nonflu,0.036213,0.036266,0.037121,0.033605


In [312]:
nationalfinindex = nationaldf[nationaldf[FinancialPredicted] > 0].index.to_list()
nationallegindex = nationaldf[nationaldf[LegalPredicted] > 0].index.to_list()
nationalhealindex = nationaldf[nationaldf[HealthPredicted] > 0].index.to_list()
nationalschindex = nationaldf[nationaldf[SchoolPredicted] > 0].index.to_list()

temp = df_new[df_new.index.isin(nationalfinindex)]
natiionalfinmean = temp.mean()
temp = df_new[df_new.index.isin(nationallegindex)]
natiionalegmean = temp.mean()
temp = df_new[df_new.index.isin(nationalhealindex)]
natiionalhealmean = temp.mean()
temp = df_new[df_new.index.isin(nationalschindex)]
natiionalschmean = temp.mean()

finaldf = pd.DataFrame()
finaldf['Category'] = natiionalfinmean.index
finaldf[FinancialLabel] = natiionalfinmean.to_list()
finaldf[Legallabel] = natiionalegmean.to_list()
finaldf[SchoolLabel] = natiionalschmean.to_list()
finaldf[HealthLabel] = natiionalhealmean.to_list()

finaldf.to_csv('national liwc scores.csv')

In [313]:
regionalfinindex = regionaldf[regionaldf[FinancialPredicted] > 0].index.to_list()
regionallegindex = regionaldf[regionaldf[LegalPredicted] > 0].index.to_list()
regionalhealindex = regionaldf[regionaldf[HealthPredicted] > 0].index.to_list()
regionalschindex = regionaldf[regionaldf[SchoolPredicted] > 0].index.to_list()

temp = df_new[df_new.index.isin(regionalfinindex)]
regiionalfinmean = temp.mean()
temp = df_new[df_new.index.isin(regionallegindex)]
regiionalegmean = temp.mean()
temp = df_new[df_new.index.isin(regionalhealindex)]
regiionalhealmean = temp.mean()
temp = df_new[df_new.index.isin(regionalschindex)]
regiionalschmean = temp.mean()

finaldf = pd.DataFrame()
finaldf['Category'] = regiionalfinmean.index
finaldf[FinancialLabel] = regiionalfinmean.to_list()
finaldf[Legallabel] = regiionalegmean.to_list()
finaldf[SchoolLabel] = regiionalschmean.to_list()
finaldf[HealthLabel] = regiionalhealmean.to_list()

finaldf.to_csv('regional liwc scores.csv')

In [314]:
finaldf = pd.DataFrame()
finaldf['Category'] = natiionalegmean.index
finaldf[FinancialLabel + " - National"] = natiionalfinmean.to_list()
finaldf[FinancialLabel + " - Regional"] = regiionalfinmean.to_list()
finaldf[Legallabel + " - National"] = natiionalegmean.to_list()
finaldf[Legallabel + " - Regional"] = regiionalegmean.to_list()
finaldf[SchoolLabel + " - National"] = natiionalschmean.to_list()
finaldf[SchoolLabel + " - Regional"] = regiionalschmean.to_list()
finaldf[HealthLabel + " - National"] = natiionalhealmean.to_list()
finaldf[HealthLabel + " - Regional"] = regiionalhealmean.to_list()

finaldf.to_csv('national vs regional liwc scores.csv')

In [315]:
leftfinindex = leftbias[leftbias[FinancialPredicted] > 0].index.to_list()
leftlegindex = leftbias[leftbias[LegalPredicted] > 0].index.to_list()
lefthealindex = leftbias[leftbias[HealthPredicted] > 0].index.to_list()
leftschindex = leftbias[leftbias[SchoolPredicted] > 0].index.to_list()

liwcresultsleftfin = df_new[df_new.index.isin(leftfinindex)]
leftfinmean = liwcresultsleftfin.mean()
liwcresultsleftleg = df_new[df_new.index.isin(leftlegindex)]
leftlegmean = liwcresultsleftleg.mean()
liwcresultsleftheal = df_new[df_new.index.isin(lefthealindex)]
lefthealmean = liwcresultsleftheal.mean()
liwcresultsleftsch = df_new[df_new.index.isin(leftschindex)]
leftschmean = liwcresultsleftsch.mean()

finaldf = pd.DataFrame()
finaldf['Category'] = leftfinmean.index
finaldf[FinancialLabel] = leftfinmean.to_list()
finaldf[Legallabel] = leftlegmean.to_list()
finaldf[SchoolLabel] = leftschmean.to_list()
finaldf[HealthLabel] = lefthealmean.to_list()

finaldf.to_csv('left bias liwc scores.csv')

In [316]:
rightfinindex = rightbias[rightbias[FinancialPredicted] > 0].index.to_list()
rightlegindex = rightbias[rightbias[LegalPredicted] > 0].index.to_list()
righthealindex = rightbias[rightbias[HealthPredicted] > 0].index.to_list()
rightschindex = rightbias[rightbias[SchoolPredicted] > 0].index.to_list()

liwcresultsrightfin = df_new[df_new.index.isin(rightfinindex)]
rightfinmean = liwcresultsrightfin.mean()
liwcresultsrightleg = df_new[df_new.index.isin(rightlegindex)]
rightlegmean = liwcresultsrightleg.mean()
liwcresultsrightheal = df_new[df_new.index.isin(righthealindex)]
righthealmean = liwcresultsrightheal.mean()
liwcresultsrightsch = df_new[df_new.index.isin(rightschindex)]
rightschmean = liwcresultsrightsch.mean()

finaldf = pd.DataFrame()
finaldf['Category'] = rightfinmean.index
finaldf[FinancialLabel] = rightfinmean.to_list()
finaldf[Legallabel] = rightlegmean.to_list()
finaldf[SchoolLabel] = rightschmean.to_list()
finaldf[HealthLabel] = righthealmean.to_list()

finaldf.to_csv('right bias liwc scores.csv')

In [317]:
finaldf = pd.DataFrame()
finaldf['Category'] = rightfinmean.index
finaldf[FinancialLabel + " - Left"] = leftfinmean.to_list()
finaldf[FinancialLabel + " - Right"] = rightfinmean.to_list()
finaldf[Legallabel + " - Left"] = leftlegmean.to_list()
finaldf[Legallabel + " - Right"] = rightlegmean.to_list()
finaldf[SchoolLabel + " - Left"] = leftschmean.to_list()
finaldf[SchoolLabel + " - Right"] = rightschmean.to_list()
finaldf[HealthLabel + " - Left"] = lefthealmean.to_list()
finaldf[HealthLabel + " - Right"] = righthealmean.to_list()

finaldf.to_csv('left vs right liwc scores.csv')

In [318]:
liwcresultsleftfin

,function,pronoun,ppron,i,we,you,shehe,they,ipron,article,...,home,money,relig,death,informal,swear,netspeak,assent,nonflu,filler
43,39.698492,9.882747,5.192630,0.000000,0.586265,2.345059,0.083752,2.177554,4.690117,5.611390,...,0.167504,0.921273,0.167504,0.586265,0.502513,0.251256,0.0,0.083752,0.167504,0.0
63,33.512545,3.763441,2.329749,0.000000,0.000000,0.000000,2.329749,0.000000,1.433692,8.243728,...,0.179211,0.358423,0.000000,2.688172,0.000000,0.000000,0.0,0.000000,0.000000,0.0
70,36.779661,5.593220,3.389831,0.000000,0.000000,0.169492,2.881356,0.338983,2.203390,7.457627,...,0.000000,0.677966,0.000000,2.372881,0.169492,0.169492,0.0,0.000000,0.000000,0.0
110,36.321627,3.789279,1.848429,0.000000,0.000000,0.000000,1.571165,0.277264,1.940850,8.595194,...,0.092421,3.419593,0.184843,0.184843,0.092421,0.000000,0.0,0.092421,0.000000,0.0
119,31.924072,3.623814,2.329594,0.000000,0.172563,0.000000,2.070751,0.086281,1.294219,7.592752,...,0.172563,0.776531,0.086281,0.086281,0.000000,0.000000,0.0,0.000000,0.000000,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8347,35.443038,10.126582,6.188467,1.969058,0.281294,0.703235,2.953586,0.281294,3.938115,3.656821,...,0.562588,0.281294,0.281294,0.421941,0.140647,0.000000,0.0,0.140647,0.000000,0.0
8429,40.358127,8.953168,3.856749,0.000000,1.377410,1.101928,0.619835,0.757576,5.027548,6.818182,...,0.344353,0.619835,0.137741,0.068871,0.000000,0.000000,0.0,0.000000,0.000000,0.0
8435,30.171073,4.354588,2.021773,0.000000,0.000000,0.000000,2.021773,0.000000,2.332815,6.376361,...,0.622084,1.710731,0.155521,0.311042,0.155521,0.000000,0.0,0.000000,0.155521,0.0
8457,40.358127,8.953168,3.856749,0.000000,1.377410,1.101928,0.619835,0.757576,5.027548,6.818182,...,0.344353,0.619835,0.137741,0.068871,0.000000,0.000000,0.0,0.000000,0.000000,0.0


In [327]:
from scipy import stats

# Example data for two independent groups
group1_data = liwcresultsleftfin['we'].to_list()
group2_data = liwcresultsrightfin['we'].to_list()

# Perform Welch's t-test
t_statistic, p_value = stats.ttest_ind(group2_data, group1_data, equal_var=False)

# Print the results
print("Welch's t-test:")
print("t-statistic:", t_statistic)
print("p-value:", p_value)

Welch's t-test:
t-statistic: 0.2135519345620527
p-value: 0.8314625828703506


In [3]:
import pandas as pd
data = pd.read_csv('final working dataset with only suicide articles.csv')
publishers = pd.read_csv('publishers_regrouped.csv')

datapubs = data['Publisher'].to_list()
allpubs = publishers['All Publishers'].to_list()

allpubs = [i.strip() for i in allpubs]
datapubs = [i.strip() for i in datapubs]

print(len(set(datapubs)))
print(len(set(allpubs)))
print(set(datapubs) - set(allpubs))

nationalpublishers = publishers['National Publishers'].dropna().to_list()
regionalpublishers = publishers['Regional Publishers'].dropna().to_list()
leftpublishers = publishers['Left Leaning Publishers'].dropna().to_list()
rightpublishers = publishers['Right Leaning Publishers'].dropna().to_list()
fringepublishers = publishers['Fringe/Questionable Publishers'].dropna().to_list()

nationalpublishers = [i.strip() for i in nationalpublishers]
regionalpublishers = [i.strip() for i in regionalpublishers]
leftpublishers = [i.strip() for i in leftpublishers]
rightpublishers = [i.strip() for i in rightpublishers]
fringepublishers = [i.strip() for i in fringepublishers]

279
271
{'bonginoreport', 'freedom-bunker', 'skynewsus', 'pamelagellerreport', 'csmonitor', 'breaking911', 'thewashingtonstandard', 'hitandrun'}
['pbs', ' thehill', ' chicagotribune', ' fortune', ' chicagosun-times', ' realclearpolitics', ' talkingpointsmemo', ' crooksandliars', ' charlotteobserver', ' dailybeast', ' nationalreview', ' usatoday', ' motherjones', ' dailyherald', ' npr', ' rawstory', ' politico', ' foxnews', ' newsweek', ' newyorker', ' usnews', ' thenewyorktimes', ' cnbc', ' democracynow', ' yahoonews', ' rightwingwatch', ' theintercept', ' theatlantic', ' abcnews', ' slate', ' msnbc', ' thehuffingtonpost', ' newyorkpost', ' cbsnews', ' washingtonpost', ' hotair', ' thedailycaller', ' vox', ' businessinsider', ' dailykos', ' buzzfeed', ' cnn', ' newyorkdailynews', ' thewashingtonexaminer', ' dailywire']


In [4]:
def left_func(row):
    val = row['Publisher']
    #print(val)
    if row['Publisher'] in leftpublishers:
        return 1
    else:
        return 0
def right_func(row):
    val = row['Publisher']
    #print(val)
    if row['Publisher'] in rightpublishers:
        return 1
    else:
        return 0
def national_func(row):
    val = row['Publisher']
    #print(val)
    if row['Publisher'] in nationalpublishers:
        return 1
    else:
        return 0
def regional_func(row):
    val = row['Publisher']
    #print(val)
    if row['Publisher'] in regionalpublishers:
        return 1
    else:
        return 0
def fringe_func(row):
    val = row['Publisher']
    #print(val)
    if row['Publisher'] in fringepublishers:
        return 1
    else:
        return 0

# Apply the function to create the new column
data['Left'] = data.apply(left_func, axis=1)
data['Right'] = data.apply(right_func, axis=1)
data['National'] = data.apply(national_func, axis=1)
data['Regional'] = data.apply(regional_func, axis=1)
data['Fringe/Questionable'] = data.apply(fringe_func, axis=1)

data.to_csv('final working dataset with only suicide articles.csv')

In [ ]:
regionaldf = data[data['Regional'] == 1]
nationaldf = data[data['National'] == 1]
rightdf = data[data['Left'] == 1]
rightdf = data[data['Right'] == 1]
fringedf = data[data['Fringe/Questionable'] == 1]

In [ ]:
print("National: ", nationaldf.shape[0])
print("Financial: ", nationaldf[nationaldf[FinancialPredicted] > 0].shape[0])
print("Legal: ", nationaldf[nationaldf[LegalPredicted] > 0].shape[0])
print("School: ", nationaldf[nationaldf[SchoolPredicted] > 0].shape[0])
print("Health: ", nationaldf[nationaldf[HealthPredicted] > 0].shape[0])
print("Not any category: ",nationaldf[nationaldf['None - Predicted'] > 0].shape[0])

In [ ]:
print("National: ", nationaldf.shape[0])
print("Financial %: ", 100 * nationaldf[nationaldf[FinancialPredicted] > 0].shape[0]/nationaldf.shape[0])
print("Legal %: ", 100 * nationaldf[nationaldf[LegalPredicted] > 0].shape[0]/nationaldf.shape[0])
print("School %: ", 100 * nationaldf[nationaldf[SchoolPredicted] > 0].shape[0]/nationaldf.shape[0])
print("Health %: ", 100 * nationaldf[nationaldf[HealthPredicted] > 0].shape[0]/nationaldf.shape[0])
print("Not any category %: ", 100 * nationaldf[nationaldf['None - Predicted'] > 0].shape[0]/nationaldf.shape[0])